In [1]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Query to get all buildings in Bac Ninh province
# The query finds all nodes (points) and ways (polygons) that are tagged as buildings
query = """
[out:json];
area["name"="Bac Ninh"]->.searchArea;
(
  node["building"](area.searchArea);
  way["building"](area.searchArea);
  relation["building"](area.searchArea);
);
out center;
"""

# Run the query
result = api.query(query)

# Extract the coordinates of buildings
buildings = []
for node in result.nodes:
    buildings.append((node.lat, node.lon))
for way in result.ways:
    if way.nodes:
        center_lat = sum(node.lat for node in way.nodes) / len(way.nodes)
        center_lon = sum(node.lon for node in way.nodes) / len(way.nodes)
        buildings.append((center_lat, center_lon))
for relation in result.relations:
    if relation.members:
        center_lat = sum(member.lat for member in relation.members if member.lat) / len(relation.members)
        center_lon = sum(member.lon for member in relation.members if member.lon) / len(relation.members)
        buildings.append((center_lat, center_lon))

# Print the coordinates
for lat, lon in buildings:
    print(f"Latitude: {lat}, Longitude: {lon}")


In [2]:
import requests
import json

def get_building_coordinates(province):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    area["name:en"="{province}"]["admin_level"="4"]->.searchArea;
    (
      way["building"](area.searchArea);
      relation["building"](area.searchArea);
    );
    out center;
    """
    
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()
    
    coordinates = []
    for element in data['elements']:
        if 'center' in element:
            lat = element['center']['lat']
            lon = element['center']['lon']
            coordinates.append((lat, lon))
    
    return coordinates

# Example usage
province = "Bac Ninh"
building_coordinates = get_building_coordinates(province)

print(f"Number of buildings found: {len(building_coordinates)}")
print("First 10 coordinates:")
for lat, lon in building_coordinates[:10]:
    print(f"Latitude: {lat}, Longitude: {lon}")

# Optionally, save to a file
with open(f"{province.replace(' ', '_')}_building_coordinates.json", 'w') as f:
    json.dump(building_coordinates, f)

Number of buildings found: 0
First 10 coordinates:


In [3]:
import requests
import json

def get_building_coordinates(province):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    area["name:en"="{province}"]["admin_level"="4"]->.searchArea;
    (
      way["building"](area.searchArea);
      relation["building"](area.searchArea);
    );
    out center;
    """
    
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()
    
    coordinates = []
    for element in data['elements']:
        if 'center' in element:
            lat = element['center']['lat']
            lon = element['center']['lon']
            coordinates.append((lat, lon))
    
    return coordinates

# Example usage
province = "Ha Noi"
building_coordinates = get_building_coordinates(province)

print(f"Number of buildings found in {province}: {len(building_coordinates)}")
print("First 10 coordinates:")
for lat, lon in building_coordinates[:10]:
    print(f"Latitude: {lat}, Longitude: {lon}")

# Optionally, save to a file
with open(f"{province.replace(' ', '_')}_building_coordinates.json", 'w') as f:
    json.dump(building_coordinates, f)

Number of buildings found in Ha Noi: 0
First 10 coordinates:


In [4]:
import requests
import json

def get_building_coordinates(province, limit=1000):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    area["name"="{province}"]->.searchArea;
    (
      way["building"](area.searchArea);
      relation["building"](area.searchArea);
    );
    out center {limit};
    """
    
    try:
        response = requests.get(overpass_url, params={'data': overpass_query}, timeout=60)
        response.raise_for_status()
        data = response.json()
        
        coordinates = []
        for element in data.get('elements', []):
            if 'center' in element:
                lat = element['center']['lat']
                lon = element['center']['lon']
                coordinates.append((lat, lon))
        
        return coordinates
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
provinces = ["Hà Nội", "Bắc Ninh"]

for province in provinces:
    building_coordinates = get_building_coordinates(province)

    print(f"\nProvince: {province}")
    print(f"Number of buildings found: {len(building_coordinates)}")
    
    if building_coordinates:
        print("First 5 coordinates:")
        for lat, lon in building_coordinates[:5]:
            print(f"Latitude: {lat}, Longitude: {lon}")

        # Optionally, save to a file
        with open(f"{province.replace(' ', '_')}_building_coordinates.json", 'w', encoding='utf-8') as f:
            json.dump(building_coordinates, f, ensure_ascii=False)
        print(f"Coordinates saved to {province.replace(' ', '_')}_building_coordinates.json")
    else:
        print("No buildings found. This could be due to:")
        print("1. The province name might be incorrect or not recognized by OpenStreetMap.")
        print("2. There might be no data for this area in OpenStreetMap.")
        print("3. The query might have timed out due to the area's size.")


Province: Hà Nội
Number of buildings found: 0
No buildings found. This could be due to:
1. The province name might be incorrect or not recognized by OpenStreetMap.
2. There might be no data for this area in OpenStreetMap.
3. The query might have timed out due to the area's size.

Province: Bắc Ninh
Number of buildings found: 0
No buildings found. This could be due to:
1. The province name might be incorrect or not recognized by OpenStreetMap.
2. There might be no data for this area in OpenStreetMap.
3. The query might have timed out due to the area's size.
